First add the data functions to the path

In [ ]:
import sys
import numpy as np
import pandas as pd
import copy
from netCDF4 import Dataset
from pathlib import Path
from datetime import datetime
notebook_dir = !pwd # Replace with !echo %cd% if using windows
library_dir = Path(notebook_dir[0]).parent.absolute()
sys.path.append(str(library_dir))
data_raw_path = library_dir / 'data' / 'raw'

from src.data.adcp_mission_analysis import adcp_import_data, add_dive_averages, read_glider_nc
#from src.data.beam_mapping import beam2enu, beam_from_center, caltran

Use better plotting defaults

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
style_path = library_dir / 'src' / 'visualization' / 'presentation.mplstyle'
style.use(str(style_path))

Get an overview of the dataset

In [ ]:
overview, mission_dict = adcp_import_data(data_raw_path)
overview

Add beam attributes. These are taken from the fifth cell (11.3 m from the glider on default settings) and averaged over each ascent/descent profile

In [ ]:
beam_attrs, adcp_df = add_dive_averages(overview, mission_dict)
beam_attrs
# overview.join(beam_attrs)

In [ ]:
beam_attrs.plot(subplots=True,figsize=(20,20),marker='o');

See which variables are correlated between the dive limbs

In [ ]:
beam_attrs.corr()

Load in glider attitude data for comparison with that from the ADCP. A mismatch may indicate physcal misalignment of the glider and ADCP or a poor calibration of the accelerometers in either unit.

In [ ]:
glider_df = read_glider_nc(Path('/home/callum/Documents/Eureka/data/glider-nc-transfer/full-nc'))

In [ ]:
fig, ax = plt.subplots(3,1,figsize=(20,26), sharex=True)
ax = ax.ravel()
ax[0].scatter(glider_df.index, glider_df.pitch_gl, label='glider')
ax[0].scatter(adcp_df.index, adcp_df.pitch_ad, label='adcp')
ax[0].set(xlim=[datetime(2020,1,24,0), datetime(2020,1,24,12)], ylim=[-30,30], ylabel='Pitch')
ax[1].scatter(glider_df.index, glider_df.roll_gl)
ax[1].scatter(adcp_df.index, adcp_df.roll_ad)
ax[1].set(ylim=[-20,20], ylabel='Roll')
ax[2].scatter(glider_df.index, glider_df.heading_gl)
ax[2].scatter(adcp_df.index, adcp_df.heading_ad)
ax[2].set(ylim=[50,200], ylabel='Heading')
ax[0].legend();

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.scatter(glider_df.index, glider_df.pressure_gl, label='glider')
ax.scatter(adcp_df.index, adcp_df.pressure_ad, label='adcp')
ax.set(xlim=[datetime(2020,1,24,0), datetime(2020,1,24,12)],ylim = [0,750], ylabel='Pressure (dbar)')
ax.legend()

After 40 dives, the glider should be trimmed and on deep dives. we will use this as a sample dive to check coordinate transforms

In [ ]:
foo = mission_dict['0041a']
bar = mission_dict['0041b']

Plot the xyz velocites from one dive cycle. Upper panel dive, lower panel climb

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(12,14), sharex=False)
ax = ax.ravel()
ax[0].scatter(-foo.glider_w_from_p,foo.vel_xyz[:,0,0][:-1], label='x')
ax[0].scatter(-foo.glider_w_from_p,foo.vel_xyz[:,0,1][:-1], label='y')
ax[0].scatter(-foo.glider_w_from_p,foo.vel_xyz[:,0,2][:-1], label='z')
ax[0].set(xlim=[0,0.2], ylim=[-0.4,0.4],ylabel='xyz water velocity from ADCP m/s')
ax[0].legend()
ax[1].scatter(-bar.glider_w_from_p,bar.vel_xyz[:,0,0][:-1], label='x')
ax[1].scatter(-bar.glider_w_from_p,bar.vel_xyz[:,0,1][:-1], label='y')
ax[1].scatter(-bar.glider_w_from_p,bar.vel_xyz[:,0,2][:-1], label='z')
ax[1].set(xlim=[-0.25,0.], ylim=[-0.4,0.4], xlabel='w from pressure m/s')

As above for enu velocites

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(12,14), sharex=False)
ax = ax.ravel()
ax[0].scatter(-foo.glider_w_from_p,foo.vel_enu[:,0,0][:-1], label='x')
ax[0].scatter(-foo.glider_w_from_p,foo.vel_enu[:,0,1][:-1], label='y')
ax[0].scatter(-foo.glider_w_from_p,foo.vel_enu[:,0,2][:-1], label='z')
ax[0].set(xlim=[0,0.2], ylim=[-0.4,0.4],ylabel='water velocity from ADCP m/s')
ax[0].legend()
ax[1].scatter(-bar.glider_w_from_p,bar.vel_enu[:,0,0][:-1], label='x')
ax[1].scatter(-bar.glider_w_from_p,bar.vel_enu[:,0,1][:-1], label='y')
ax[1].scatter(-bar.glider_w_from_p,bar.vel_enu[:,0,2][:-1], label='z')
ax[1].set(xlim=[-0.25,0.], ylim=[-0.4,0.4], xlabel='w from pressure m/s')

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(foo.vel_xyz[:,0,0][:-1],foo.vel_xyz[:,0,1][:-1], label='dive')
ax.scatter(bar.vel_xyz[:,0,0][:-1],bar.vel_xyz[:,0,1][:-1], label='climb')
#ax.scatter(-bar.glider_w_from_p,bar.vel_xyz[:,0,1][:-1], label='y')
#ax.scatter(-bar.glider_w_from_p,bar.vel_xyz[:,0,2][:-1], label='z')
ax.set(xlim=[-0.4,0.4], ylim=[-0.4,0.4], xlabel='x vel (m/s)',ylabel='y vel (m/s)')
ax.legend()

In [ ]:
print(len(foo.heading))
print(len(foo.vel_enu[:,0,0][:-1]))

In [ ]:
fig, ax = plt.subplots(2,1, sharex=True, figsize=(12,8))
ax[0].scatter(foo.heading[:-1], foo.vel_enu[:,0,0][:-1], label='dive eastward')
ax[0].scatter(foo.heading[:-1], foo.vel_enu[:,0,1][:-1], label='dive northward')
ax[1].scatter(bar.heading[:-1], bar.vel_enu[:,0,0][:-1], label='climb eastward')
ax[1].scatter(bar.heading[:-1], bar.vel_enu[:,0,1][:-1], label='climb northward')
ax[1].set(xlabel='Heading from North',ylabel='velocity (m/s)')
ax[1].legend()
ax[0].legend();

In [ ]:
mens = []
mene = []
menn = []
menu = []
for ident in list(mission_dict):
    mens.append(np.nanmedian(np.abs(mission_dict[ident].vel_enu)))
    mene.append(np.nanmedian(mission_dict[ident].vel_enu[:,:,0]))
    menn.append(np.nanmedian(mission_dict[ident].vel_enu[:,:,1]))
    menu.append(np.nanmedian(mission_dict[ident].vel_enu[:,:,2]))

In [ ]:
xens = []
xene = []
xenn = []
xenu = []
for ident in list(mission_dict):
    xens.append(np.nanmedian(np.abs(mission_dict[ident].vel_xyz)))
    xene.append(np.nanmedian(mission_dict[ident].vel_xyz[:,:,0]))
    xenn.append(np.nanmedian(mission_dict[ident].vel_xyz[:,:,1]))
    xenu.append(np.nanmedian(mission_dict[ident].vel_xyz[:,:,2]))

In [ ]:
bens = []
bene = []
benn = []
benu = []
for ident in list(mission_dict):
    bens.append(np.nanmedian(np.abs(mission_dict[ident].vel_beam)))
    bene.append(np.nanmedian(mission_dict[ident].vel_beam[:,:,0]))
    benn.append(np.nanmedian(mission_dict[ident].vel_beam[:,:,1]))
    benu.append(np.nanmedian(mission_dict[ident].vel_beam[:,:,2]))

Median beam velocites on descent beams 2,3,4 and ascent beams 1,2,4

In [ ]:
dx = []
dy = []
dz = []
cx = []
cy = []
cz = []
for ident in list(mission_dict):
    if ident[-1] == 'a':
        dx.append(np.nanmedian(mission_dict[ident].vel_beam[:,:,0]))
        dy.append(np.nanmedian(mission_dict[ident].vel_beam[:,:,1]))
        dz.append(np.nanmedian(mission_dict[ident].vel_beam[:,:,2]))
    else:
        cx.append(np.nanmedian(mission_dict[ident].vel_beam[:,:,0]))
        cy.append(np.nanmedian(mission_dict[ident].vel_beam[:,:,1]))
        cz.append(np.nanmedian(mission_dict[ident].vel_beam[:,:,2]))

In [ ]:
print(np.nanmedian(dx))
print(np.nanmedian(dy))
print(np.nanmedian(dz))
print(np.nanmedian(cx))
print(np.nanmedian(cy))
print(np.nanmedian(cz))

median speed relative to xyz on descent and climb

In [ ]:
dx = []
dy = []
dz = []
cx = []
cy = []
cz = []
for ident in list(mission_dict):
    if ident[-1] == 'a':
        dx.append(np.nanmedian(mission_dict[ident].vel_xyz[:,:,0]))
        dy.append(np.nanmedian(mission_dict[ident].vel_xyz[:,:,1]))
        dz.append(np.nanmedian(mission_dict[ident].vel_xyz[:,:,2]))
    else:
        cx.append(np.nanmedian(mission_dict[ident].vel_xyz[:,:,0]))
        cy.append(np.nanmedian(mission_dict[ident].vel_xyz[:,:,1]))
        cz.append(np.nanmedian(mission_dict[ident].vel_xyz[:,:,2]))

In [ ]:
print(np.nanmedian(dx))
print(np.nanmedian(dy))
print(np.nanmedian(dz))
print(np.nanmedian(cx))
print(np.nanmedian(cy))
print(np.nanmedian(cz))

Average velocities in xyz and enu

In [ ]:
print(np.nanmedian(np.abs(xene)))
print(np.nanmedian(np.abs(xenn)))
print(np.nanmedian(np.abs(xenu)))
print(np.nanmedian(np.abs(mene)))
print(np.nanmedian(np.abs(menn)))
print(np.nanmedian(np.abs(menu)))

xyz velocites over the mission

In [ ]:
fig, ax = plt.subplots(3,1,figsize=(20,20), sharex=True)
ax = ax.ravel()
#ax[0].scatter(mene,menn,label='all', c=beam_attrs['pitch'], cmap='RdBu',vmin=-30, vmax=30)
ax[0].scatter(np.arange(len(mene)),xene,label='east', c=beam_attrs['pitch'], cmap='RdBu',vmin=-30, vmax=30)
ax[1].scatter(np.arange(len(mene)),xenn,label='north', c=beam_attrs['pitch'], cmap='RdBu',vmin=-30, vmax=30)
ax[2].scatter(beam_attrs['heading'],xenu,label='up', c=beam_attrs['pitch'], cmap='RdBu',vmin=-30, vmax=30)
ax[0].set(ylim=[-0.3,0.3],ylabel='along glider vel')
ax[1].set(ylim=[-0.3,0.3], ylabel='across vel')
ax[2].set(ylim=[-0.3,0.3],ylabel='upward of glider vel', xlim=[0,360], xlabel='Heading from North');

enu velocities over the mission

In [ ]:
fig, ax = plt.subplots(3,1,figsize=(20,20), sharex=True)
ax = ax.ravel()
#ax[0].scatter(mene,menn,label='all', c=beam_attrs['pitch'], cmap='RdBu',vmin=-30, vmax=30)
ax[0].scatter(np.arange(len(mene)),mene,label='east', c=beam_attrs['pitch'], cmap='RdBu',vmin=-30, vmax=30)
ax[1].scatter(np.arange(len(mene)),menn,label='north', c=beam_attrs['pitch'], cmap='RdBu',vmin=-30, vmax=30)
ax[2].scatter(beam_attrs['heading'],menu,label='up', c=beam_attrs['pitch'], cmap='RdBu',vmin=-30, vmax=30)
ax[0].set(ylim=[-0.3,0.3],ylabel='eastward vel')
ax[1].set(ylim=[-0.3,0.3], ylabel='northward vel')
ax[2].set(ylim=[-0.3,0.3],ylabel='upward vel', xlim=[0,360], xlabel='Heading from North');